# Custom Seeds with Watershed Algorithm

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
road = cv2.imread("../imgs/road_image.jpg")

In [3]:
road_copy = np.copy(road)

In [4]:
road.shape

(600, 800, 3)

In [5]:
road.shape[0:2]

(600, 800)

In [6]:
marker_image = np.zeros(shape=road.shape[0:2], dtype=np.int32)

In [7]:
marker_image.shape

(600, 800)

In [8]:
segments = np.zeros(road.shape,dtype=np.uint8)

In [9]:
segments.shape

(600, 800, 3)

#### Choosing Color Mapping

In [10]:
from matplotlib import cm

In [11]:
cm.tab10(0)

(0.12156862745098039, 0.4666666666666667, 0.7058823529411765, 1.0)

In [12]:
np.array(cm.tab10(0)[:3])*255

array([ 31., 119., 180.])

In [13]:
tuple(np.array(cm.tab10(0)[0:3])*255)

(31.0, 119.0, 180.0)

In [14]:
def create_rgb(i):
    return tuple(np.array(cm.tab10(i)[0:3])*255)

In [15]:
colors = []

for i in range(10):
    colors.append(create_rgb(i))

In [16]:
colors

[(31.0, 119.0, 180.0),
 (255.0, 127.0, 14.0),
 (44.0, 160.0, 44.0),
 (214.0, 39.0, 40.0),
 (148.0, 103.0, 189.0),
 (140.0, 86.0, 75.0),
 (227.0, 119.0, 194.0),
 (127.0, 127.0, 127.0),
 (188.0, 189.0, 34.0),
 (23.0, 190.0, 207.0)]

#### Creating Mouse Callback

In [17]:
# Numbers 0-9
n_markers = 10

In [18]:
# Default settings
current_marker = 1
marks_updated = False

In [19]:
def mouse_callback(event, x, y, flags, param):
    global marks_updated 

    if event == cv2.EVENT_LBUTTONDOWN:
        
        # TRACKING FOR MARKERS
        cv2.circle(marker_image, (x, y), 10, (current_marker), -1)
        
        # DISPLAY ON USER IMAGE
        cv2.circle(road_copy, (x, y), 10, colors[current_marker], -1)
        marks_updated = True


In [20]:
# Loop
cv2.namedWindow("Road Image")
cv2.setMouseCallback("Road Image", mouse_callback)

while True:
    
    cv2.imshow("Watershed Segments", segments)
    cv2.imshow("Road Image", road_copy)
    
    
    k = cv2.waitKey(1)
    
    if k == 27:
        break
        
    # CLEARING ALL THE COLORS PRESS C KES
    elif k == ord("c"):
        road_copy = road.copy()
        marker_image = np.zeros(road.shape[0:2],dtype=np.int32)
        segments = np.zeros(road.shape,dtype=np.uint8)
        
    # UPDATE COLOR CHOICE
    elif k > 0 and chr(k).isdigit():
        current_marker = int(chr(k))
        
    
    # UPDATE THE MARKINGS
    if marks_updated:
        
        marker_image_copy = marker_image.copy()
        cv2.watershed(road,marker_image_copy)
        
        segments = np.zeros(road.shape,dtype=np.uint8)
        
        for color_ind in range(n_markers):
            segments[marker_image_copy==(color_ind)] = colors[color_ind]
        
        
        
cv2.destroyAllWindows()